In [1]:
# coding:utf-8


import pickle
import gensim
from gensim import corpora, models
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
import mecab_util
import warnings
warnings.filterwarnings('ignore')
from gensim.models import CoherenceModel
import pprint

In [2]:
TOPIC_NUM = 20 # if  None: coherenceで選択

SEPARATE_COMMENT_TO_ONE = False # コメントも1文ずつに分ける

COMMENT_ROUGE = False # コメントとのROUGEを計算するか

SELECT_SENTENCE_NUM = None # if None: 3割抽出

USE_MALLET = False # model: malletを使用するか

In [3]:
def load_pkl(path):
    with open(path, "rb") as f:
        data = pickle.load(f)
    return data

def fix_text(text):
    text = text.replace('\n', ' ')
    text = text.replace('\u3000', ' ')
    text = text.replace('!', ' ')
    text = text.replace('！', ' ')
    text = text.replace('?', ' ')
    text = text.replace('？', ' ')
    text = text.replace('.', ' ')
    text = text.replace('。', ' ')
    return text

def load_title(articles, target_url):
    article_title = articles[target_url][1]
    return article_title

def load_article_text(articles, target_url):
    article_text = articles[target_url][-1]
    article_text = "。".join(fix_text(article_text).split())
    return article_text

def load_article_comments(articles, target_url):
    file_name = "./comments/" + articles[target_url][0].split("/")[-1][5:] + ".p"
    comments = load_pkl(file_name)
    # user, date, comment, agree, disagree
    comment_texts = [i["comment"] for i in comments]
    return comment_texts

def separate_comment2sentence(comment_texts):
    all_sentences = []# 各コメントも一文ずつに分ける
    for c in comment_texts:
        for s in c.split():
            all_sentences.append(s)
    return all_sentences

def remove_outlier_sentence(sentences, mecabed_list, minimum=5, maximum=20):
    # 短い文章を削除
    new_sentences = []
    new_mecabed_list = []
    for s, m in zip(sentences, mecabed_list):
        if len(m) < minimum or len(m) > maximum:
            pass
        else:
            new_sentences.append(s)
            new_mecabed_list.append(m)
    return new_sentences, new_mecabed_list

def format_topics_sentences(ldamodel, corpus, texts, topic_num):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    # Perc_Contribution: 指定された文書のトピックの寄与率

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

def top_topic_sentences(df_topic_sents_keywords):
    # Group top 5 sentences under each topic
    sent_topics_sorteddf = pd.DataFrame()
    sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')
    
    for i, grp in sent_topics_outdf_grpd:
        sent_topics_sorteddf = pd.concat([sent_topics_sorteddf, grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], axis=0)
    
    # Reset Index    
    sent_topics_sorteddf.reset_index(drop=True, inplace=True)
    # Format
    sent_topics_sorteddf.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]
    return sent_topics_sorteddf

def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.LdaModel(corpus=corpus, id2word=dictionary,num_topics=num_topics, random_state = 1)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [4]:
target = "https://headlines.yahoo.co.jp/hl?a=20181118-00000030-asahi-soci"
#target = "https://headlines.yahoo.co.jp/hl?a=20181128-00000013-exmusic-musi"

#target = "https://headlines.yahoo.co.jp/hl?a=20181118-00000530-san-soci"
#target = "https://headlines.yahoo.co.jp/hl?a=20181122-00010002-mrt-l45"
#target = "https://headlines.yahoo.co.jp/hl?a=20181122-00010002-mrt-l45"
#target = "https://headlines.yahoo.co.jp/hl?a=20181123-35129123-cnn-int"

#target = "https://headlines.yahoo.co.jp/hl?a=20181119-00010010-abema-soci" #ない
#target = "https://headlines.yahoo.co.jp/hl?a=20181122-00000081-reut-bus_all"
#target = "https://headlines.yahoo.co.jp/hl?a=20181119-00025348-mbsnewsv-l28'"

In [5]:
articles = load_pkl("./yahoo_news_url.p")

In [6]:
title = load_title(articles, target)
print("title: ", title)

title:  留学生の在留資格審査が厳格に　日本への門、一気に狭く（朝日新聞デジタル） - Yahoo!ニュース


In [7]:
article_text = load_article_text(articles, target)
#print("本文: \n", article_text)
#print("文字数: ", len(article_text))
print("文書数: ", len(article_text.split("。")))
print(article_text)

文書数:  10
東京入国管理局が、日本語学校などで学ぶため来日する留学生の在留資格審査を厳格化させていることが、関係者の話で分かった。不法就労などへの対策強化が背景にあるが、東南アジアなど一部の国では在留資格の交付率が激減し、日本留学への門が一気に閉ざされた形となっている。【写真】外国人労働者に関する野党合同ヒアリングに出席し、思いを述べた後、涙を拭う技能実習生。法務省によると、留学生は２０１７年末で国内に約３１万人おり、５年前から１０万人以上増えている。一方、就労目的の来日も指摘され、在留資格が取り消される例も出ている。政府は現在、外国人労働者受け入れ拡大のため、新たな在留資格を盛り込んだ出入国管理法改正案の成立を目指している。成立した場合は就労目的の外国人に新たな道が開ける一方、留学希望者は減る可能性がある。留学生は主に４月と１０月に入学するため、在留資格審査の時期が集中する。東京入管が日本語学校向けにまとめた資料によると、今年の４月期は２万９８０１件の申請があり、７７・７％が交付されたが、１０月期は１万６４９５件の申請に対し、６５・６％の交付にとどまった。数値には一部の専門学校などのデータが含まれていないという


In [8]:
sentences = article_text.split("。")
if len(sentences) < 10:
    print("sentencesが10未満")

In [9]:
raw_comments = load_article_comments(articles, target)
comment_texts = [fix_text(i) for i in raw_comments]

In [10]:
if SEPARATE_COMMENT_TO_ONE is True:
    comment_sentences = separate_comment2sentence(comment_texts)# コメントも1文ずつに分ける
else:
    comment_sentences = comment_texts
pprint.pprint(comment_sentences[:10])

['もっと厳しくして欲しい  不良外国人を入れたら治安も悪化するし、将来的に外国人生活保護も激増して日本の社会保障は崩壊します ',
 'どう厳格化されたのかが分からない  この内容じゃ、厳格化というより申請が集中するから審査が追い付いていないだけみたいな感じ',
 'ルールを 厳格化することじゃない ルール自体が的得てないんだ ルールであってルールで無してっか事だと思う 法整備に 的得た、ポリシー理念が何もないんだ '
 'だから 夜盗 としては、そこを突いてくる 当然な流れ イタチごっこだろ',
 '厳格化ではなく今までがゆるゆるだったのではないか ',
 '狭くなる、という言い回しは印象操作  法の適用を厳密化した、で十分   不法滞在につながることを阻止しようというもの  '
 '日本語ができない人を労働者として受け入れるというのは理解できない  '
 '日本語ができるようにと考えると、どうしても日本語と語順が近い韓国になってしまい、これは後の憂いの元になりかねない  '
 '企業が労働条件や賃金を上げていけば、人は集まるだろうに  '
 '女性が家に入っても十分大丈夫という環境も作っていかないと、いつまでたっても子どもの数は増えていかない ',
 'これもそうですが、最近の国会の議論は十分な議論もなくなし崩し的に審議が進められて、「ハイ、○○時間審議したので（強行）採決します」となっている '
 '（採決の際、議論の中身がどうこうはあまり論じられない ） '
 '今後何らかの問題が生じたときに「やはりこうすべきでした」の反省ではなく、「だってあの時皆さんで決めたことでしょう '
 '」と政府与党側が言い逃れをするようなことにならないでほしい  切に願う ',
 '特に、韓国から入ってくる人間は厳格に扱うべきだ ',
 '在留資格の付与に関しては、政策目標を明確にし、過去の制度運営状況を観察しながら、担当行政庁の裁量権を広く認める方向で考えなければならない   '
 'その目標に関しては、日本に骨を埋める人材、日本社会・日本文化を尊重する人材、日本を侵害しようとする勢力に対して日本の為に戦う人材を獲得するということを、是非とも明確に掲げてもらいたい   '
 'そういう人材であれば、低賃金で雇うことなど、違法であり、決して許されないことが明確になる  違法企

In [11]:
#from sklearn.model_selection import train_test_split
#comment_sentences, test_comment_sentences = train_test_split(comment_sentences, test_size = 0.2)

In [12]:
mecabed_list, nouns_dict = mecab_util.basic_process(comment_sentences)

comment_sentences, mecabed_list = remove_outlier_sentence(comment_sentences, mecabed_list)

In [13]:
print("使えるコメント数: ", len(mecabed_list))
pprint.pprint(comment_sentences[:10])

使えるコメント数:  505
['もっと厳しくして欲しい  不良外国人を入れたら治安も悪化するし、将来的に外国人生活保護も激増して日本の社会保障は崩壊します ',
 'どう厳格化されたのかが分からない  この内容じゃ、厳格化というより申請が集中するから審査が追い付いていないだけみたいな感じ',
 'ルールを 厳格化することじゃない ルール自体が的得てないんだ ルールであってルールで無してっか事だと思う 法整備に 的得た、ポリシー理念が何もないんだ '
 'だから 夜盗 としては、そこを突いてくる 当然な流れ イタチごっこだろ',
 '特に、韓国から入ってくる人間は厳格に扱うべきだ ',
 '人数を絞るより悪徳仲介業者を取り除く方が双方のため',
 '現在の日本語専門学校の留学生には、問題が多すぎる  留学生とは名ばかりで不法就労の温床であり、学校側も審査、立ち入り を厳格すべきである ',
 '人が足りていないのはわかるが、このままを維持するために誰彼侵入させるのは本当にどうかと思う '
 '国としてどうありたいのか、何を大事に未来に向かうのか、きちんとロードマップやビジョンを国民に示すべき 少なくとも今は不法滞在などの問題は解決すべき ',
 '警察も日本語が話せないという理由で、大柄な態度の外国人に甘かったり、見て見ぬふりするのももう少し取締を強化してほしい ',
 'さらなる厳格化をおねがいしたいです  ベトナムや、中国を受け入れるデメリットの方がはるかに大きい 明らかに悪いことする目的な奴が増えている ',
 '日本に来るのは本当に真面目に仕事をする人なのか勉強ができて優秀な人かこのふたパターンでいいと思う  '
 '仕事をするか見極めるには1、2年ベトナムで働かせてみて気持ちがぶれなかったら来させればいいと思う だいたいボロがでるからね   やっぱね '
 '安易にいれるべきやないと思うよ ']


In [14]:
pprint.pprint(mecabed_list[:10])

[['厳しく', '不良外国人', '治安', '悪化', '将来的', '外国人生活保護', '激増', '日本', '社会保障', '崩壊'],
 ['厳格', '分から', '内容', '厳格化', 'いう', '申請', '集中', '審査', '追い付い'],
 ['ルール',
  '厳格',
  'ルール',
  '自体',
  '得て',
  'ルール',
  'ルール',
  '無し',
  'てっか',
  '法整備',
  '得',
  'ポリシー',
  '理念',
  '夜盗',
  '突い',
  '流れ',
  'イタチごっこ'],
 ['韓国', '入っ', '人間', '厳格', '扱う'],
 ['人数', '絞る', '悪徳', '仲介業者', '取り除く', '双方'],
 ['現在',
  '日本語',
  '専門学校',
  '留学生',
  '多',
  '留学生',
  '名ばかり',
  '不法就労',
  '温床',
  '学校',
  '側',
  '審査',
  '立ち入り',
  '厳格'],
 ['足り',
  'このまま',
  '維持',
  '誰彼',
  '侵入',
  'せる',
  '国',
  '未来',
  '向かう',
  'ロードマップ',
  'ビジョン',
  '国民',
  '示す',
  '不法滞在',
  '解決'],
 ['警察', '日本語', '話せ', '大柄', '態度', '外国人', '甘かっ', '取締', '強化'],
 ['厳格化',
  'おねがい',
  'ベトナム',
  '中国',
  '受け入れる',
  'デメリット',
  'はるか',
  '大きい',
  '明らか',
  '悪いこと',
  '目的',
  '増え'],
 ['日本',
  '真面目',
  '優秀',
  'パターン',
  '見極める',
  '2年',
  'ベトナム',
  '働かせ',
  'ぶれ',
  '来さ',
  'せれ',
  'ボロ',
  'でる',
  '安易',
  'いれる',
  'ないと']]


In [15]:
dictionary = gensim.corpora.Dictionary(mecabed_list)
# 文章ごとの単語のリストから辞書を作る

dictionary.filter_extremes(no_below=2, no_above=0.1)
#dictionary.filter_extremes(no_below=1)
#dictionary.filter_extremes(no_below=5, no_above=0.2)
# no_berow: 使われてる文章がno_berow個以下の単語無視
# no_above: 使われてる文章の割合がno_above以上の場合無視
print("辞書数: ", len(dictionary))

辞書数:  679


In [16]:
corpus = [dictionary.doc2bow(doc) for doc in mecabed_list]

In [17]:
pprint.pprint(corpus[:10])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)],
 [(6, 1), (7, 1), (8, 1), (9, 1), (10, 1)],
 [(11, 4), (12, 1), (13, 1), (14, 1), (15, 1)],
 [(16, 1), (17, 1), (18, 1)],
 [(19, 1), (20, 1), (21, 1)],
 [(9, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1)],
 [(31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1)],
 [(28, 1), (41, 1), (42, 1), (43, 1), (44, 1)],
 [(8, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1)],
 [(46, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1)]]


In [18]:
# Can take a long time to run.
if TOPIC_NUM is None:
    model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=mecabed_list, start=3, limit=60, step=3)
    max_index = coherence_values.index(max(coherence_values))
    limit=60; start=3; step=3;
    x = range(start, limit, step)
    topic_num = x[max_index]
else:
    topic_num = TOPIC_NUM

print("トピック数: ", topic_num)

トピック数:  20


In [19]:
if TOPIC_NUM is None:
    # Print the coherence scores
    for m, cv in zip(x, coherence_values):
        print("Num Topics =", m, " has Coherence Value of", round(cv, 4))
        
     # Show graph
    limit=60; start=3; step=3;
    x = range(start, limit, step)
    plt.plot(x, coherence_values)
    plt.xlabel("Num Topics")
    plt.ylabel("Coherence score")
    plt.legend(("coherence_values"), loc='best')
    plt.show()

In [20]:
# 昔の
#tfidf = gensim.models.TfidfModel(corpus)
#corpus_tfidf = tfidf[corpus]
#lda = gensim.models.LdaModel(corpus=corpus_tfidf, id2word=dictionary,num_topics=topic_num, random_state = 1)

if USE_MALLET is False:
    lda = gensim.models.LdaModel(corpus=corpus, id2word=dictionary,num_topics=topic_num, random_state = 1)
else:
    mallet_path = './mallet-2.0.8/bin/mallet'
    lda = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, id2word=dictionary, num_topics=topic_num)

In [21]:
for i in range(topic_num):
    print('tpc_{0}: {1}'.format(i, lda.print_topic(i)[0:100]+'...'))

tpc_0: 0.024*"法律" + 0.024*"文句" + 0.024*"中国人" + 0.016*"形" + 0.016*"就労" + 0.016*"審査" + 0.016*"登録" + 0.008*"経団...
tpc_1: 0.028*"入国" + 0.024*"審査" + 0.020*"記事" + 0.016*"すれ" + 0.016*"賛成" + 0.016*"安倍" + 0.012*"国外追放" + 0.012*"...
tpc_2: 0.026*"目的" + 0.021*"しろ" + 0.017*"ビザ" + 0.016*"大勢" + 0.016*"対応" + 0.016*"日本人" + 0.015*"労働" + 0.013*"入...
tpc_3: 0.044*"日本語学校" + 0.027*"なっ" + 0.024*"大学" + 0.020*"学生" + 0.017*"目的" + 0.017*"日本人" + 0.017*"審査" + 0.017...
tpc_4: 0.034*"目的" + 0.025*"日本語学校" + 0.025*"入国" + 0.017*"受け入れ" + 0.017*"審査" + 0.017*"不法滞在" + 0.017*"野放し" + 0...
tpc_5: 0.023*"記事" + 0.016*"結婚" + 0.016*"入っ" + 0.016*"分から" + 0.016*"示し" + 0.016*"関係者" + 0.016*"犯罪" + 0.016*"...
tpc_6: 0.031*"犯罪" + 0.025*"ルール" + 0.019*"なっ" + 0.019*"基準" + 0.016*"学生" + 0.013*"移民" + 0.013*"中国" + 0.013*"減...
tpc_7: 0.032*"日本語" + 0.022*"入国" + 0.016*"ベトナム" + 0.016*"日本人" + 0.016*"帰化" + 0.012*"朝日新聞" + 0.011*"朝鮮" + 0.0...
tpc_8: 0.024*"就労" + 0.018*"処罰" + 0.018*"大半" + 0.018*"先" + 0.017*"ビザ" + 0.015*"犯罪" + 0.012*"目的" + 0.012*"ベトナ...
t

In [22]:
for i in range(topic_num):
    print(lda.print_topic(i))

0.024*"法律" + 0.024*"文句" + 0.024*"中国人" + 0.016*"形" + 0.016*"就労" + 0.016*"審査" + 0.016*"登録" + 0.008*"経団連" + 0.008*"大学" + 0.008*"難民"
0.028*"入国" + 0.024*"審査" + 0.020*"記事" + 0.016*"すれ" + 0.016*"賛成" + 0.016*"安倍" + 0.012*"国外追放" + 0.012*"不法滞在" + 0.012*"厳格化" + 0.012*"治安"
0.026*"目的" + 0.021*"しろ" + 0.017*"ビザ" + 0.016*"大勢" + 0.016*"対応" + 0.016*"日本人" + 0.015*"労働" + 0.013*"入れる" + 0.011*"日本国内" + 0.011*"不法就労"
0.044*"日本語学校" + 0.027*"なっ" + 0.024*"大学" + 0.020*"学生" + 0.017*"目的" + 0.017*"日本人" + 0.017*"審査" + 0.017*"就労ビザ" + 0.014*"学校" + 0.012*"言っ"
0.034*"目的" + 0.025*"日本語学校" + 0.025*"入国" + 0.017*"受け入れ" + 0.017*"審査" + 0.017*"不法滞在" + 0.017*"野放し" + 0.017*"使う" + 0.017*"外国人労働者" + 0.017*"拡大"
0.023*"記事" + 0.016*"結婚" + 0.016*"入っ" + 0.016*"分から" + 0.016*"示し" + 0.016*"関係者" + 0.016*"犯罪" + 0.016*"人たち" + 0.016*"口座" + 0.008*"人手不足"
0.031*"犯罪" + 0.025*"ルール" + 0.019*"なっ" + 0.019*"基準" + 0.016*"学生" + 0.013*"移民" + 0.013*"中国" + 0.013*"減ら" + 0.013*"先" + 0.013*"自体"
0.032*"日本語" + 0.022*"入国" + 0.016*"ベトナム" + 0.016*"日本人" + 0.016*"帰化" + 

In [23]:
if USE_MALLET is False:
    # Compute Perplexity
    print('Perplexity: ', lda.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
    
    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda, texts=mecabed_list, dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('Coherence Score: ', coherence_lda)

Perplexity:  -7.6318626000982315
Coherence Score:  0.5695345664916243


In [24]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
3     -0.040094 -0.098827       1        1  8.583449
12     0.006253 -0.107789       2        1  6.513201
1      0.069435 -0.041399       3        1  6.445032
18    -0.007014 -0.074922       4        1  6.257403
16     0.064508 -0.066888       5        1  5.894820
9     -0.030648  0.022279       6        1  5.854464
10     0.094260 -0.025363       7        1  5.789567
15     0.162595  0.029689       8        1  5.505211
13    -0.072702  0.058328       9        1  5.435796
2     -0.117036 -0.052410      10        1  5.352506
7      0.116871  0.109858      11        1  5.035540
14    -0.076694  0.157081      12        1  4.773398
17    -0.042652 -0.050888      13        1  4.587060
6     -0.016709 -0.000952      14        1  4.463264
8      0.001233  0.054688      15        1  4.217972
5     -0.016617  0.102308      16        1  3.397980
0     -0.027769 -0.032019      17        1  3.227507
11     0.010906  0.034541      18        1  3.187230
4      0.029449 -0.034636      19        1  3.066853
19    -0.107575  0.017320      20        1  2.411750, topic_info=     Category       Freq    Term      Total  loglift  logprob
term                                                         
192   Default  23.000000   日本語学校  23.000000  30.0000  30.0000
398   Default  15.000000     韓国人  15.000000  29.0000  29.0000
9     Default  34.000000      審査  34.000000  28.0000  28.0000
247   Default  21.000000     中国人  21.000000  27.0000  27.0000
204   Default  17.000000      狭く  17.000000  26.0000  26.0000
33    Default  13.000000    不法滞在  13.000000  25.0000  25.0000
11    Default   8.000000     ルール   8.000000  24.0000  24.0000
35    Default  16.000000      国民  16.000000  23.0000  23.0000
28    Default  14.000000     日本語  14.000000  22.0000  22.0000
78    Default  16.000000      就労  16.000000  21.0000  21.0000
52    Default  38.000000      目的  38.000000  20.0000  20.0000
236   Default  22.000000      大学  22.000000  19.0000  19.0000
95    Default  26.000000      入国  26.000000  18.0000  18.0000
126   Default  12.000000      記事  12.000000  17.0000  17.0000
91    Default  22.000000      犯罪  22.000000  16.0000  16.0000
150   Default  15.000000    在留資格  15.000000  15.0000  15.0000
173   Default   7.000000      異常   7.000000  14.0000  14.0000
177   Default  21.000000      学生  21.000000  13.0000  13.0000
299   Default  29.000000     日本人  29.000000  12.0000  12.0000
46    Default   9.000000    ベトナム   9.000000  11.0000  11.0000
113   Default  13.000000  外国人労働者  13.000000  10.0000  10.0000
111   Default  21.000000    受け入れ  21.000000   9.0000   9.0000
122   Default  13.000000      移民  13.000000   8.0000   8.0000
47    Default  10.000000      中国  10.000000   7.0000   7.0000
117   Default  32.000000      なっ  32.000000   6.0000   6.0000
106   Default  15.000000      外人  15.000000   5.0000   5.0000
138   Default   8.000000      法律   8.000000   4.0000   4.0000
48    Default  14.000000   受け入れる  14.000000   3.0000   3.0000
76    Default  22.000000      ビザ  22.000000   2.0000   2.0000
139   Default  12.000000    生活保護  12.000000   1.0000   1.0000
...       ...        ...     ...        ...      ...      ...
173   Topic20   2.365297      異常   7.261445   2.6031  -3.4276
244   Topic20   0.814283      早急   3.144111   2.3738  -4.4940
500   Topic20   0.814283      生活   3.145639   2.3734  -4.4940
222   Topic20   0.814283      指紋   3.172052   2.3650  -4.4940
611   Topic20   0.814283      罰金   3.192752   2.3585  -4.4940
472   Topic20   0.814283      犯し   3.199253   2.3565  -4.4940
5     Topic20   0.814283    社会保障   3.219055   2.3503  -4.4940
634   Topic20   0.814283      技術   3.242347   2.3431  -4.4940
83    Topic20   0.814283    健康保険   3.319468   2.3196  -4.4940
107   Topic20   1.589790      子供   7.279597   2.2033  -3.8249
223   Topic20   0.814283      登録   3.936561   2.1491  -4.4940
418   Topic20   0.814283      結婚   3.955926   2.1442  -4.4940
3

In [25]:
# 各文章のトピック
df_topic_sents_keywords = format_topics_sentences(lda, corpus, comment_sentences, topic_num)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,14.0,0.8643,"犯罪, 大学, 生活保護, 措置, 増え, 優秀, 日本語, 日本人, せる, 社会",もっと厳しくして欲しい 不良外国人を入れたら治安も悪化するし、将来的に外国人生活保護も激増...
1,1,1.0,0.8417,"入国, 審査, 記事, すれ, 賛成, 安倍, 国外追放, 不法滞在, 厳格化, 治安",どう厳格化されたのかが分からない この内容じゃ、厳格化というより申請が集中するから審査が追...
2,2,6.0,0.8944,"犯罪, ルール, なっ, 基準, 学生, 移民, 中国, 減ら, 先, 自体",ルールを 厳格化することじゃない ルール自体が的得てないんだ ルールであってルールで無してっ...
3,3,17.0,0.7625,"移民, ビザ, 言っ, 働き, 入国, 日本人, 厳格化, 目的, なっ, 韓国",特に、韓国から入ってくる人間は厳格に扱うべきだ
4,4,7.0,0.7625,"日本語, 入国, ベトナム, 日本人, 帰化, 朝日新聞, 朝鮮, 方々, 移民, 内部",人数を絞るより悪徳仲介業者を取り除く方が双方のため
5,5,10.0,0.9136,"狭く, 韓国人, 学生, 朝日, ベトナム, 大学, 中国人, いら, 中国, 多",現在の日本語専門学校の留学生には、問題が多すぎる 留学生とは名ばかりで不法就労の温床であり...
6,6,12.0,0.9136,"国民, 外人, 受け入れ, 増える, 学校, 狭く, 日本人, 中国人, 治安, 法律",人が足りていないのはわかるが、このままを維持するために誰彼侵入させるのは本当にどうかと思う ...
7,7,7.0,0.8417,"日本語, 入国, ベトナム, 日本人, 帰化, 朝日新聞, 朝鮮, 方々, 移民, 内部",警察も日本語が話せないという理由で、大柄な態度の外国人に甘かったり、見て見ぬふりするのももう...
8,8,7.0,0.9050,"日本語, 入国, ベトナム, 日本人, 帰化, 朝日新聞, 朝鮮, 方々, 移民, 内部",さらなる厳格化をおねがいしたいです ベトナムや、中国を受け入れるデメリットの方がはるかに大...
9,9,14.0,0.8944,"犯罪, 大学, 生活保護, 措置, 増え, 優秀, 日本語, 日本人, せる, 社会",日本に来るのは本当に真面目に仕事をする人なのか勉強ができて優秀な人かこのふたパターンでいいと...


In [26]:
#　各トピックで最も寄与率が高い文章
top_topic_df = top_topic_sentences(df_topic_sents_keywords)
top_topic_df

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.9136,"法律, 文句, 中国人, 形, 就労, 審査, 登録, 経団連, 大学, 難民",やっと着手したかと言う感想 留学生の肩書き持った不法就労者や犯罪者が あまりにも多すぎる ...
1,1.0,0.9367,"入国, 審査, 記事, すれ, 賛成, 安倍, 国外追放, 不法滞在, 厳格化, 治安",入口を狭めるのと同時に、入国後の監視/フォローも強化して欲しい 入国さえ出来れば後はユル...
2,2.0,0.9321,"目的, しろ, ビザ, 大勢, 対応, 日本人, 労働, 入れる, 日本国内, 不法就労",就学ビザで来た人が働ける現行制度がそもそも異常 諸外国並みに労働ビザと就学ビザをきっちり分け...
3,3.0,0.9321,"日本語学校, なっ, 大学, 学生, 目的, 日本人, 審査, 就労ビザ, 学校, 言っ",厳格化というより適正化したんだろ いままで就労目的の留学生もどんどん入れてきたけど、学生は...
4,4.0,0.9321,"目的, 日本語学校, 入国, 受け入れ, 審査, 不法滞在, 野放し, 使う, 外国人労働者...",いや、本来の目的ではない入国を野放しにしておく方が問題だ 正しい手続きで入国したな人への差別...
5,5.0,0.9208,"記事, 結婚, 入っ, 分から, 示し, 関係者, 犯罪, 人たち, 口座, 人手不足",アメリカでの留学ビザの場合、 相応の残高証明が求められます 留学が目的ですから就労しなく...
6,6.0,0.9367,"犯罪, ルール, なっ, 基準, 学生, 移民, 中国, 減ら, 先, 自体",労働の禁止をしてやったら罰金刑におけばいい 他の記事では除染させられたとか書いてたけど、何...
7,7.0,0.9321,"日本語, 入国, ベトナム, 日本人, 帰化, 朝日新聞, 朝鮮, 方々, 移民, 内部",今迄が物凄く緩すぎただけ 在留資格審査を厳格化しても、 その分、簡単に永住できるようにするの...
8,8.0,0.9208,"就労, 処罰, 大半, 先, ビザ, 犯罪, 目的, ベトナム人, 居る, 日本人",入国させてからも定期的にイレギュラーで様子を見に行き、 現実に則してなければ即帰国させるべき...
9,9.0,0.9367,"入国, 東南アジア, 日本人, ビザ, 普通に, 犯罪, 目的, 海外, おかしい, 持っ",同時に新大久保辺りの東南アジア系のお店の取締りも強化してください 観光ビザで不法侵入してその...


In [27]:
sentences = article_text.split("。")
mecabed_article, _ = mecab_util.basic_process(sentences)

In [28]:
pprint.pprint(mecabed_article[:10])

[['東京入国管理局', '日本語学校', '学ぶ', '来日', '留学生', '在留資格', '審査', '厳格', '関係者', '分かっ'],
 ['不法就労',
  '対策',
  '強化',
  '背景',
  '東南アジア',
  '一部',
  '国',
  '在留資格',
  '交付',
  '率',
  '激減',
  '日本',
  '留学',
  '門',
  '閉ざさ',
  '形',
  'なっ'],
 ['写真', '外国人労働者', '野党', '合同', 'ヒアリング', '出席', '述べ', '涙', '拭う', '技能実習生'],
 ['法務省', 'よる', '留学生', '末', '国内', '増え'],
 ['就労', '目的', '来日', '指摘', '在留資格', '取り消さ', '例'],
 ['政府',
  '現在',
  '外国人労働者',
  '受け入れ',
  '拡大',
  '新た',
  '在留',
  '資格',
  '盛り込ん',
  '出入国管理法',
  '改正',
  '案',
  '成立',
  '目指し'],
 ['成立', '就労', '目的', '外国人', '新た', '道', '開ける', '留学', '希望者', '減る', '可能性'],
 ['留学生', '主', '１０月', '入学', '在留資格', '審査', '時期', '集中'],
 ['入管',
  '日本語学校',
  '向け',
  '資料',
  'よる',
  '今年',
  '期',
  '件',
  '申請',
  '交付',
  '１０月',
  '期',
  '件',
  '申請',
  '交付',
  'とどまっ'],
 ['数値', '一部', '専門学校', 'データ', '含ま']]


In [29]:
if SELECT_SENTENCE_NUM is None:
    select_sentence_num = int(len(sentences) / 3) # 文書数の何パーセントか
else:
    select_sentence_num = SELECT_SENTENCE_NUM
    
print("要約の文章数: ", select_sentence_num)

要約の文章数:  3


In [30]:
# 既存の辞書を使用して、コーパスを作成
test_corpus = [dictionary.doc2bow(text) for text in mecabed_article]
#test_corpus

In [31]:
def topic_getter(lda, corpus):
    topics = []
    for topics_per_document in lda[corpus]:
        #print(topics_per_document)
        sentence_sorted_topic = sorted(topics_per_document, key=(lambda x: x[1]), reverse=True)
        #print("sorted: ", sentence_sorted_topic)
        topics.append(sentence_sorted_topic[0])
    return topics

In [32]:
topics = topic_getter(lda, test_corpus)
pprint.pprint(topics)

[(3, 0.3857888),
 (15, 0.6204229),
 (12, 0.5690769),
 (18, 0.45376083),
 (15, 0.81000006),
 (18, 0.34223214),
 (16, 0.37530875),
 (18, 0.3679972),
 (15, 0.6898951),
 (8, 0.68333334)]


In [33]:
order_topics = [(t[0], t[1], i) for i, t in enumerate(topics)]
pprint.pprint(order_topics)

[(3, 0.3857888, 0),
 (15, 0.6204229, 1),
 (12, 0.5690769, 2),
 (18, 0.45376083, 3),
 (15, 0.81000006, 4),
 (18, 0.34223214, 5),
 (16, 0.37530875, 6),
 (18, 0.3679972, 7),
 (15, 0.6898951, 8),
 (8, 0.68333334, 9)]


In [34]:
def kiyoritu(order_topics, select_sentence_num, sentences):
    """ 寄与率のみに着目 """
    sort_topics = sorted(order_topics, key=lambda order_topics: order_topics[1], reverse=True)
    top = []
    for i in sort_topics[:select_sentence_num]:
        top.append(i[2])
    top = sorted(top)
    summary = []
    for i in top:
        summary.append(sentences[i])
    return summary 

def topic_count(order_topics, df_dominant_topic, select_sentence_num, sentences, reverse=False):
    """ コメントのトピック数をカウント """
    topic_counter = {}
    for i in df_dominant_topic["Dominant_Topic"]:
        if i not in topic_counter:
            topic_counter[i] = 1
        else:
            topic_counter[i] += 1

    if reverse is False:
        topic_counter_list = sorted(topic_counter, key=topic_counter.get, reverse=True)
    else:
        topic_counter_list = sorted(topic_counter, key=topic_counter.get)

    top = []
    for c in topic_counter_list:
        for o in order_topics:
            sentence_topic = o[0]
            if sentence_topic == c:
                top.append(o[2])
    top = top[:select_sentence_num]
    top = sorted(top)
    summary = []
    for i in top:
        summary.append(sentences[i])
    return summary

def different_topic(order_topics, select_sentence_num, sentences):
    """ なるべく異なるトピックを抽出 """
    li = [i[0] for i in order_topics]
    import collections
    c = collections.Counter(li)
    #print(c.most_common()[::-1])
    top = []
    for i in c.most_common()[::-1]:
        top.append(i[0])
    top = top[:select_sentence_num]
    top = sorted(top)
    summary = []
    for i in top:
        for o in order_topics:
            if o[0] == i:
                summary.append(sentences[o[2]])
    return summary

In [35]:
topic1 = kiyoritu(order_topics, select_sentence_num, sentences)
topic2 = topic_count(order_topics, df_dominant_topic, select_sentence_num, sentences)
topic3 = topic_count(order_topics, df_dominant_topic, select_sentence_num, sentences, True)
topic4 = different_topic(order_topics, select_sentence_num, sentences)

## 生成した要約1

In [36]:
print("。\n\n".join(topic1))

一方、就労目的の来日も指摘され、在留資格が取り消される例も出ている。

東京入管が日本語学校向けにまとめた資料によると、今年の４月期は２万９８０１件の申請があり、７７・７％が交付されたが、１０月期は１万６４９５件の申請に対し、６５・６％の交付にとどまった。

数値には一部の専門学校などのデータが含まれていないという


## 生成した要約2

In [37]:
print("。\n\n".join(topic2))

東京入国管理局が、日本語学校などで学ぶため来日する留学生の在留資格審査を厳格化させていることが、関係者の話で分かった。

法務省によると、留学生は２０１７年末で国内に約３１万人おり、５年前から１０万人以上増えている。

成立した場合は就労目的の外国人に新たな道が開ける一方、留学希望者は減る可能性がある


## 生成した要約 3

In [38]:
print("。\n\n".join(topic3))

不法就労などへの対策強化が背景にあるが、東南アジアなど一部の国では在留資格の交付率が激減し、日本留学への門が一気に閉ざされた形となっている。

一方、就労目的の来日も指摘され、在留資格が取り消される例も出ている。

東京入管が日本語学校向けにまとめた資料によると、今年の４月期は２万９８０１件の申請があり、７７・７％が交付されたが、１０月期は１万６４９５件の申請に対し、６５・６％の交付にとどまった


## 生成した要約 4

In [39]:
print("。\n\n".join(topic4))

数値には一部の専門学校などのデータが含まれていないという。

【写真】外国人労働者に関する野党合同ヒアリングに出席し、思いを述べた後、涙を拭う技能実習生。

成立した場合は就労目的の外国人に新たな道が開ける一方、留学希望者は減る可能性がある


In [40]:
import lexrank
lex =  lexrank.summarize(article_text, sent_limit=select_sentence_num)

## LexRank

In [41]:
print("\n\n".join(lex))

東京入国管理局が、日本語学校などで学ぶため来日する留学生の在留資格審査を厳格化させていることが、関係者の話で分かった。

不法就労などへの対策強化が背景にあるが、東南アジアなど一部の国では在留資格の交付率が激減し、日本留学への門が一気に閉ざされた形となっている。

留学生は主に４月と１０月に入学するため、在留資格審査の時期が集中する。


In [42]:
import random
index = random.sample(range(len(article_text.split("。"))), select_sentence_num)
index = sorted(index)
test = []
for i in range(select_sentence_num):
    test.append(article_text.split("。")[index[i]])

## random

In [43]:
print(index)
print("。\n\n".join(test))

[2, 5, 9]
【写真】外国人労働者に関する野党合同ヒアリングに出席し、思いを述べた後、涙を拭う技能実習生。

政府は現在、外国人労働者受け入れ拡大のため、新たな在留資格を盛り込んだ出入国管理法改正案の成立を目指している。

数値には一部の専門学校などのデータが含まれていないという


In [44]:
print("title: ", title)
#print("本文: \n", article_text)
#print("文字数: ", len(article_text))
print("文書数: ", len(article_text.split("。")))
print("使えるコメント数: ", len(mecabed_list))
print("辞書数: ", len(dictionary))
print("トピック数: ", topic_num)
print("要約の文章数: ", select_sentence_num)

title:  留学生の在留資格審査が厳格に　日本への門、一気に狭く（朝日新聞デジタル） - Yahoo!ニュース
文書数:  10
使えるコメント数:  505
辞書数:  679
トピック数:  20
要約の文章数:  3


In [45]:
result = []
result.append(title)
result.append(len(article_text.split("。")))
result.append(len(mecabed_list))

In [46]:
from sumeval.metrics.rouge import RougeCalculator
rouge = RougeCalculator(stopwords=True, lang="ja")
def cal_rouge(summary, ref):
    rouge_1 = rouge.rouge_n(summary=summary, references=ref, n=1)
    rouge_2 = rouge.rouge_n(summary=summary, references=ref, n=2)
    return rouge_1, rouge_2

In [47]:
lex_summary = "".join(lex)

In [48]:
# randomとlexのrougeを計算
test_summary = "。".join(test) + "。"
r1, r2 = cal_rouge(test_summary, lex_summary)
result.append(r1)
result.append(r2)

In [49]:
# topic1とlexのrougeを計算
topic1_summary = "。".join(topic1) + "。"
r1, r2 = cal_rouge(topic1_summary, lex_summary)
result.append(r1)
result.append(r2)

In [50]:
# topic2とlexのrougeを計算
topic2_summary = "。".join(topic2) + "。"
r1, r2 = cal_rouge(topic2_summary, lex_summary)
result.append(r1)
result.append(r2)

In [51]:
# topic3とlexのrougeを計算
topic3_summary = "。".join(topic3) + "。"
r1, r2 = cal_rouge(topic3_summary, lex_summary)
result.append(r1)
result.append(r2)

In [52]:
# topic4とlexのrougeを計算
topic4_summary = "。".join(topic4) + "。"
r1, r2 = cal_rouge(topic4_summary, lex_summary)
result.append(r1)
result.append(r2)

In [53]:
result_df = pd.DataFrame()
result_df = result_df.append(pd.Series([result[i] for i in range(1, 13)]), ignore_index=True)

In [54]:
result_df.columns = ['本文の文書数', 'コメント数','random_r1', 'random_r2', '寄与率_r1', '寄与率_r2', "topic-count_r1", "topic-count_r2", "topic-count-reverse_r1", "topic-count-reverse_r2", "topic-least_r1", "topic-least_r2"]

In [55]:
result_df

,本文の文書数,コメント数,random_r1,random_r2,寄与率_r1,寄与率_r2,topic-count_r1,topic-count_r2,topic-count-reverse_r1,topic-count-reverse_r2,topic-least_r1,topic-least_r2
0,10.0,505.0,0.293706,0.042553,0.425287,0.081395,0.584416,0.368421,0.612245,0.42268,0.316547,0.014599


In [56]:
test_summary

'【写真】外国人労働者に関する野党合同ヒアリングに出席し、思いを述べた後、涙を拭う技能実習生。政府は現在、外国人労働者受け入れ拡大のため、新たな在留資格を盛り込んだ出入国管理法改正案の成立を目指している。数値には一部の専門学校などのデータが含まれていないという。'

In [57]:
lex_summary

'東京入国管理局が、日本語学校などで学ぶため来日する留学生の在留資格審査を厳格化させていることが、関係者の話で分かった。不法就労などへの対策強化が背景にあるが、東南アジアなど一部の国では在留資格の交付率が激減し、日本留学への門が一気に閉ざされた形となっている。留学生は主に４月と１０月に入学するため、在留資格審査の時期が集中する。'

In [58]:
topic1_summary

'一方、就労目的の来日も指摘され、在留資格が取り消される例も出ている。東京入管が日本語学校向けにまとめた資料によると、今年の４月期は２万９８０１件の申請があり、７７・７％が交付されたが、１０月期は１万６４９５件の申請に対し、６５・６％の交付にとどまった。数値には一部の専門学校などのデータが含まれていないという。'

In [59]:
topic2_summary

'東京入国管理局が、日本語学校などで学ぶため来日する留学生の在留資格審査を厳格化させていることが、関係者の話で分かった。法務省によると、留学生は２０１７年末で国内に約３１万人おり、５年前から１０万人以上増えている。成立した場合は就労目的の外国人に新たな道が開ける一方、留学希望者は減る可能性がある。'

In [60]:
topic3_summary

'不法就労などへの対策強化が背景にあるが、東南アジアなど一部の国では在留資格の交付率が激減し、日本留学への門が一気に閉ざされた形となっている。一方、就労目的の来日も指摘され、在留資格が取り消される例も出ている。東京入管が日本語学校向けにまとめた資料によると、今年の４月期は２万９８０１件の申請があり、７７・７％が交付されたが、１０月期は１万６４９５件の申請に対し、６５・６％の交付にとどまった。'